In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchvision.models import densenet121, DenseNet121_Weights

# =========================================
# Device
# =========================================
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# =========================================
# Paths
# =========================================
train_dir = "/kaggle/input/datasets/puneet6060/intel-image-classification/seg_train"
test_dir  = "/kaggle/input/datasets/puneet6060/intel-image-classification/seg_test"

IMG_SIZE = 224
BATCH_SIZE = 32

# =========================================
# Transforms
# =========================================
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset  = datasets.ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("Classes:", train_dataset.classes)
num_classes = len(train_dataset.classes)
print("Number of classes:", num_classes)

# =========================================
# Load Pretrained DenseNet121
# =========================================
model = densenet121(weights=DenseNet121_Weights.DEFAULT)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Modify classifier (IMPORTANT)
in_features = model.classifier.in_features

model.classifier = nn.Sequential(
    nn.Linear(in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, num_classes)
)

model = model.to(DEVICE)

# =========================================
# Loss & Optimizer (Transfer Learning)
# =========================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# =========================================
# Training Function
# =========================================
def train_epoch(loader):
    model.train()
    total_loss = 0
    correct = 0

    for images, labels in loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels)

    acc = correct.double()/len(loader.dataset)
    return total_loss/len(loader), acc

def evaluate(loader):
    model.eval()
    correct = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)

    acc = correct.double()/len(loader.dataset)
    return acc

# =========================================
# Transfer Learning Phase
# =========================================
EPOCHS = 3
print("\nTransfer Learning...")
for epoch in range(EPOCHS):
    loss, acc = train_epoch(train_loader)
    print(f"Epoch {epoch+1}: Train Acc = {acc:.4f}")

# =========================================
# Fine-Tuning Phase
# =========================================
print("\nFine Tuning...")

# unfreeze last dense block
for name, param in model.named_parameters():
    if "denseblock4" in name:
        param.requires_grad = True

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

for epoch in range(2):
    loss, acc = train_epoch(train_loader)
    print(f"FineTune Epoch {epoch+1}: Train Acc = {acc:.4f}")

# =========================================
# Test Accuracy
# =========================================
test_acc = evaluate(test_loader)
print("\nFinal Test Accuracy:", test_acc)

torch.save(model.state_dict(), "/kaggle/working/densenet_intel.pth")
print("Model Saved!")

Device: cuda
Classes: ['seg_train']
Number of classes: 1
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


100%|██████████| 30.8M/30.8M [00:00<00:00, 189MB/s]



Transfer Learning...
Epoch 1: Train Acc = 1.0000
Epoch 2: Train Acc = 1.0000
Epoch 3: Train Acc = 1.0000

Fine Tuning...
FineTune Epoch 1: Train Acc = 1.0000
FineTune Epoch 2: Train Acc = 1.0000

Final Test Accuracy: tensor(1., device='cuda:0', dtype=torch.float64)
Model Saved!
